In [1]:
import os
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.insert(1, '/home/kseuro/Kai/deeplearnphysics/pytorch/particle_generator/')

# My stuff
import ae
import conv_ae
import utils
from dataloader import LArCV_loader

In [3]:
# Set the GPU (GPU 1 is the best option)
device = torch.device(2)

In [13]:
# Set the root path of the AutoEncoder experiments folder
exp_root = "/media/hdd1/kai/particle_generator/experiments/"

In [14]:
# Set the model type folder (either MLP or Conv model)
model_folder = "larcv_ae/" + "conv_ae/"

In [15]:
# Update the exp_root
exp_root += model_folder

## Get the names of all the experiments in the exp_root folder

In [16]:
exp_paths = []
for path in os.listdir(exp_root):
    exp_paths.append(os.path.join(exp_root, path))

print("-"*60)
for i in range(len(exp_paths)):
    print("\n Exp_{}:".format(str(i)), exp_paths[i], '\n')
    print("-"*60)

------------------------------------------------------------

 Exp_0: /media/hdd1/kai/particle_generator/experiments/larcv_ae/conv_ae/conv_ae_64_1000-epochs 

------------------------------------------------------------


In [18]:
# Set the dir of the particular experiment to be deployed
exp_dir = exp_paths[0]

In [19]:
# Create the full path to the experiment
exp_path = os.path.join(exp_root, exp_dir) + "/"
print("Experiment path set as: \n{}".format(exp_path))

Experiment path set as: 
/media/hdd1/kai/particle_generator/experiments/larcv_ae/conv_ae/conv_ae_64_1000-epochs/


In [20]:
# Load the config csv as a dict
config_csv = exp_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [21]:
# Get the model architecture from config df
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].values.item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].values.item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].values.item())**2
depth    = int(config_df[config_df['Unnamed: 0'].str.contains("depth")==True]['0'].values.item())
im_dim   = int(np.sqrt(im_size))

In [22]:
# Path to model weights
weights_dir = "weights/"

## Path to the test data

In [23]:
test_data = "/media/hdd1/kai/particle_generator/larcv_data/test/larcv_png_{}/".format(im_dim)
num_test_ex = sum( [len(examples) for _, _, examples in os.walk(test_data)] )
print("{} test data will be loaded from: \n{}".format(num_test_ex, test_data))

10000 test data will be loaded from: 
/media/hdd1/kai/particle_generator/larcv_data/test/larcv_png_64/


## Path to reference training data

In [24]:
# Path to the reference training data
train_data = "/media/hdd1/kai/particle_generator/larcv_data/train/train_reference/larcv_png_{}/".format(im_dim)
num_train_ex = sum( [len(examples) for _, _, examples in os.walk(train_data)] )
print("{} reference training data will be loaded from: \n{}".format(num_train_ex, train_data))

10000 reference training data will be loaded from: 
/media/hdd1/kai/particle_generator/larcv_data/train/train_reference/larcv_png_64/


## Set up the torch dataloader

In [25]:
loader_kwargs = {'num_workers' : 2, 'batch_size': 1, 'shuffle': True}
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5],[0.5])])
test_dataset    = LArCV_loader(root = test_data,  transforms = transforms)
train_dataset   = LArCV_loader(root = train_data, transforms = transforms) 

Image conversion flag is: L
Images will be loaded from subfolder of: /media/hdd1/kai/particle_generator/larcv_data/test/larcv_png_64/
Image conversion flag is: L
Images will be loaded from subfolder of: /media/hdd1/kai/particle_generator/larcv_data/train/train_reference/larcv_png_64/


In [26]:
test_loader = DataLoader(test_dataset, **loader_kwargs)
train_loader = DataLoader(train_dataset, **loader_kwargs)

## Setup AE layers

In [27]:
# Set up AE layer sizes
if 'mlp' in exp_path:
    base = [256] 

    # Compute encoder sizes
    sizes = lambda: [ (yield 2**i) for i in range(n_layers) ]
    enc_sizes = base * n_layers
    enc_sizes = [a*b for a,b in zip(enc_sizes, [*sizes()])][::-1]

    # Update kwarg dicts
    # Decoder is the reverse of the encoder
    ae_kwargs = {'enc_sizes' : enc_sizes, 'l_dim' : l_dim, 'im_size' : im_size, 'dec_sizes' : enc_sizes[::-1]}
else:
    depth   = [depth] * n_layers
    divisor = lambda: [ (yield 2**i) for i in range(n_layers) ]
    depth   = [a//b for a,b in zip(depth, [*divisor()])][::-1]
    ae_kwargs = {'enc_depth' : depth, 'dec_depth' : depth[::-1] }

## Setup the model loss function

In [28]:
loss_fn = nn.MSELoss().to(device)

## Load model checkpoint

In [30]:
# Get checkpoint name(s)
checkpoint_path  = exp_path + weights_dir
checkpoint_names = []
for file in os.listdir(checkpoint_path):
    checkpoint_names.append(os.path.join(checkpoint_path, file))

In [57]:
checkpoint_name_labels = []

In [60]:
print("-"*60)
for i in range(len(checkpoint_names)):
    name = checkpoint_names[i].split('/')[-1]
    checkpoint_name_labels.append(name)
    print("\n{}:".format(str(i)), name, '\n')
    print("-"*60)

------------------------------------------------------------

0: best_conv_ae_ep_650.tar 

------------------------------------------------------------

1: best_conv_ae_ep_600.tar 

------------------------------------------------------------

2: best_conv_ae_ep_999.tar 

------------------------------------------------------------

3: best_conv_ae_ep_700.tar 

------------------------------------------------------------

4: best_conv_ae_ep_850.tar 

------------------------------------------------------------

5: best_conv_ae_ep_900.tar 

------------------------------------------------------------

6: best_conv_ae_ep_950.tar 

------------------------------------------------------------

7: best_conv_ae_ep_750.tar 

------------------------------------------------------------

8: best_conv_ae_ep_800.tar 

------------------------------------------------------------


In [62]:
index = 1
current_checkpoint = checkpoint_names[index]
current_checkpoint_label = checkpoint_name_labels[index]

In [40]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(current_checkpoint)

In [41]:
# Set up model on GPU
if 'mlp' in exp_path:
    model = ae.AutoEncoder(**ae_kwargs).to(device)
else:
    model = conv_ae.ConvAutoEncoder(**ae_kwargs).to(device)

In [42]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [43]:
# Put the model in training mode
model.train()

ConvAutoEncoder(
  (encoder): ConvEncoder(
    (conv_blocks): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (3): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dila

## Loop over the test data and record the losses for the checkpoint

In [44]:
## DO NOT RE-RUN THIS CELL ##

# Keep track of MSE results for train and test set for each checkpoint
MSE_results = []

## DO NOT RE-RUN THIS CELL ##

In [45]:
test_losses = []

In [46]:
for idx, image in enumerate(test_loader):
    
    # Flatten image into a vector
    if 'mlp' in exp_path:
        image = image.view(loader_kwargs['batch_size'], -1).to(device)
    else:
        image = image.to(device)
    
    # Forward pass
    output = model(image)
    
    # Get the loss value for the batch
    loss = loss_fn(output, image)
    
    # Append loss value
    test_losses.append(float(loss.item()))

## Compute the average loss over segments of the test_loss results

In [53]:
avg_loss = []

In [54]:
step_size = 100
for step in range(0, len(test_losses), step_size):
    avg_loss.append( np.mean(test_losses[step:step+step_size]) )

## Append the average losses for checkpoint to MSE_results list

In [63]:
MSE_results.append( (current_checkpoint_label, avg_loss) )

In [65]:
len(MSE_results)

('best_conv_ae_ep_600.tar',
 [0.051372345606796446,
  0.04990461527602747,
  0.05336467694956809,
  0.0590628065308556,
  0.0529610536666587,
  0.048975902246311306,
  0.05392821523826569,
  0.060506708920001984,
  0.049383576116524636,
  0.03905495669227094,
  0.0521879989723675,
  0.0531745119066909,
  0.04021610344294459,
  0.057216570707969365,
  0.04706878937315196,
  0.048417820539325473,
  0.054501020684838294,
  0.05520246678031981,
  0.04814299096353352,
  0.05329641030519269,
  0.04740869264584035,
  0.05108458469854668,
  0.050948390811681746,
  0.059271647478453815,
  0.04571080099674873,
  0.05269901586929336,
  0.050862777039874345,
  0.05327076094923541,
  0.051250235582701864,
  0.05340486356988549,
  0.05113966939970851,
  0.04561732641654089,
  0.05431932720821351,
  0.05425816401140764,
  0.05528076795861125,
  0.05117770564742386,
  0.04302377099636942,
  0.05232137248385697,
  0.060067517287097874,
  0.05726561586372554,
  0.050156978766899556,
  0.0441620988352224

## Loop over the training data and record the average loss for the checkpoint
- Here, we only loop over the REFERENCE training examples

In [ ]:
train_losses = []

In [ ]:
for idx, image in enumerate(train_loader):    

    # Flatten image into a vector
    if 'mlp' in exp_path:
        image = image.view(loader_kwargs['batch_size'], -1).to(device)
    else:
        image = image.to(device)
    
    # Forward pass
    output = model(image)
    
    # Get the loss value for the batch
    loss = loss_fn(output, image)
    
    # Append loss value
    train_losses.append(float(loss.item()))
    
    if (idx + 1) == num_test_ex:
        break

In [ ]:
avg_train_loss = np.mean(train_losses)

In [ ]:
print("Average train loss: {} ".format( round(avg_train_loss, 3) ) )

## Find the difference between the test loss and train loss

In [ ]:
delta_MSE = abs( avg_train_loss - avg_test_loss )

In [ ]:
print("delta_MSE: {}".format( round(delta_MSE, 3)))
print('For checkpoint: \n{}'.format(current_checkpoint))

## Append the results to list

In [ ]:
results = [avg_test_loss, avg_train_loss, delta_MSE, current_checkpoint]
MSE_results.append(results)

In [ ]:
## Checkpoint Analysis
for i in range(len(MSE_results)):
    print("delta_MSE: {}".format(round(MSE_results[i][2], 5)))

## Save evaluation results to experiment folder

In [ ]:
eval_file = exp_path + "checkpoint_evaluation.txt"

In [ ]:
with open(eval_file, 'w+') as file_object:
    for exp in MSE_results:
        test_loss  = exp[0]
        train_loss = exp[1]
        delta_MSE  = exp[2]
        checkpoint = exp[3]
        line1 = "Checkpoint: {}\n".format(checkpoint)
        line2 = "test_loss: {}, train_loss: {}, delta_MSE: {}".format(test_loss, train_loss, delta_MSE)
        line = line1 + line2 + "\n" # Punctuation with a newline character is a Unix best practice
        file_object.write(line)